# Find precipitation value for pulsed and upslope storms

In [ ]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')



In [ ]:
m = ['11', '12', '01', '02', '03']
station = 'Haukeliseter'

In [ ]:
savefig = 0
if savefig == 1:
    figdir = '../../Figures/Weathermast_MEPS_Retrieval/%s/precip_seasonal_stat' %(station)
    cF.createFolder('%s/' %figdir)
form = 'png'

### Minuelty data

In [ ]:
obs_f = dict()
obs_double_fence_acc_mean = dict()
obs_wind_dir_10m   = dict()
obs_wind_speed_10m = dict()
obs_dofe_acc   = dict()
obs_wind_dir   = dict()
obs_wind_speed = dict() 
obs_dofe_acc_rolling = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_diff = dict()
obs_westerly_idx = dict()
obs_easterly_idx = dict()
obs_westerly = dict()
obs_easterly = dict()
obs_total_precip = dict()
obs_precip_west  = dict()
obs_precip_east  = dict()
obs_double_fence_temp = dict()
obs_air_temp = dict()
obs_air_temp_rolling = dict()

obs_diff_2deg = dict()
obs_total_precip_2deg = dict()
obs_precip_east_2deg = dict()
obs_precip_west_2deg = dict()

for month in m:
    if month == '11':
        t = np.arange(7,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        obs_dirnc = '../../Data/Weathermast/Met_%s%s%s_%s.nc' %(year,month,day,station)
        obs_f[int(year+month+day)] = netCDF4.Dataset(obs_dirnc, 'r')

        obs_double_fence_acc_mean[int(year+month+day)] = (np.nanmean(obs_f[int(year+month+day)].variables['double_fence_accum'][:], axis=1))
        obs_wind_dir_10m[int(year+month+day)]   = (obs_f[int(year+month+day)].variables['10m_wind_dir_mast1'][:])
        obs_wind_speed_10m[int(year+month+day)] = (obs_f[int(year+month+day)].variables['10m_wind_speed_mast1'][:])
        obs_double_fence_temp[int(year+month+day)] = (obs_f[int(year+month+day)].variables['air_temp'][:])
        
        obs_f[int(year+month+day)].close
        
        ## Create a rolling mean to smoothen out the data
        # attach the last 9 minutes from the previous day to calculate the rolling mean every 10 minutes
        if month != '11' and day == '01':
            if month == '12':
                previous_day = '30'
            elif month == '01' or month == '02':
                previous_day = '31'
            elif month == '03':
                previous_day = '28'
            if month == '01':
                previous_month = '12'
                previous_year = str(int(year)-1)
            elif month == '12':
                previous_month = str(int(month)-1)
                previous_year = year
            else:
                previous_month = '0%s' %(int(month)-1)
                previous_year = year
            
        elif int(day)-1 < 10:
            previous_day = '0%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        else:
            previous_day = '%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        if year == '2016' and month == '11' and day == '07':
            continue
        else:
            pp = (np.concatenate((obs_double_fence_acc_mean[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_acc_mean[int(year+month+day)][:])))
            obs_dofe_acc[int(year+month+day)] = (pp)
            wd = (np.concatenate((obs_wind_dir_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_dir_10m[int(year+month+day)][:])))
            obs_wind_dir[int(year+month+day)] = (wd)
            ws = (np.concatenate((obs_wind_speed_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_speed_10m[int(year+month+day)][:])))
            obs_wind_speed[int(year+month+day)] = (ws)
            at = (np.concatenate((obs_double_fence_temp[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_temp[int(year+month+day)][:])))
            obs_air_temp[int(year+month+day)] = at
            
        # Calculate the moving average. That is, take
        # the first ten values, average them, 
        # then drop the first and add the eleventh, etc.
        obs_dofe_acc[int(year+month+day)] = pd.DataFrame(np.array(obs_dofe_acc[int(year+month+day)])[:])
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc[int(year+month+day)].rolling(window=10,
                                                                                              min_periods= 7,
                                                                                              axis=0).mean()
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc_rolling[int(year+month+day)].values
        obs_dofe_acc_rolling[int(year+month+day)] = np.array(obs_dofe_acc_rolling[int(year+month+day)])[9:,:]

        obs_wind_dir[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_dir[int(year+month+day)])[:])
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir[int(year+month+day)].rolling(window=10,
                                                                                              min_periods=7,
                                                                                              axis=0).mean()
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir_rolling[int(year+month+day)].values
        obs_wind_dir_rolling[int(year+month+day)] = np.array(obs_wind_dir_rolling[int(year+month+day)])[9:,:]

        obs_wind_speed[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_speed[int(year+month+day)])[:])
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed_rolling[int(year+month+day)].values
        obs_wind_speed_rolling[int(year+month+day)] = np.array(obs_wind_speed_rolling[int(year+month+day)])[9:,:]
        
        obs_air_temp[int(year+month+day)] = pd.DataFrame(np.array(obs_air_temp[int(year+month+day)])[:])
        obs_air_temp_rolling[int(year+month+day)] = obs_air_temp[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_air_temp_rolling[int(year+month+day)] = obs_air_temp_rolling[int(year+month+day)].values
        obs_air_temp_rolling[int(year+month+day)] = np.array(obs_air_temp_rolling[int(year+month+day)])[9:,:]
        
        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        obs_diff[int(year+month+day)] = np.empty(shape=(obs_dofe_acc_rolling[int(year+month+day)].shape))#/60))
        obs_diff[int(year+month+day)][:] =  np.nan

        for i in np.arange(0,obs_dofe_acc_rolling[int(year+month+day)].shape[0]-1):#/60):
            obs_diff[int(year+month+day)][int(i),:] = obs_dofe_acc_rolling[int(year+month+day)][int(i)+1] - \
                                                      obs_dofe_acc_rolling[int(year+month+day)][int(i)]


        # Find wind regimes and associated precip amount
        IDX = np.arange(obs_wind_dir_rolling[int(year+month+day)].shape[0])

        ## find all obs_westerly regimes (202.5 - 22.5 deg) 
        obs_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 202.5), 
                                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <= 360)),
                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <22.5))]
        ## find all obs_easterly regimes (22.5 - 202.5 deg)
        obs_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 22.5),
                                                                (obs_wind_dir_rolling[int(year+month+day)][:,0] < 202.5))]

        ## find indices for all obs_westerly regimes 
        sub_idx = []

        for i in range(0,obs_westerly_idx[int(year+month+day)] .shape[0]-1):
            if obs_westerly_idx[int(year+month+day)] [i]+1 == obs_westerly_idx[int(year+month+day)] [i+1]:
                continue
            else:
                #print(i)
                sub_idx.append(i)
    
    #        print([int(year+month+day)], obs_westerly_idx[int(year+month+day)], sub_idx)
        
        
        ## find westerlies which are < 29 min and assign them to obs_easterly wind regime
        if len(obs_westerly_idx[int(year+month+day)]) == 0 and len(sub_idx) == 0:     # westerlies do not exist
            obs_westerly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]
#            print('No obs_westerly wind on %s' %(int(year+month+day)))
        elif len(obs_westerly_idx[int(year+month+day)]) != 0 and len(sub_idx) == 0:   # westerlies exist only for one time period a day
            obs_westerly[int(year+month+day)] = []
      #      obs_easterly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]
            
            obs_westerly_duration = obs_westerly_idx[int(year+month+day)][-1] - obs_westerly_idx[int(year+month+day)][0]
            if obs_westerly_duration >= 28:
                obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)],
                                                                obs_westerly_idx[int(year+month+day)][:]))
            elif obs_westerly_duration < 28:
                obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                obs_westerly_idx[int(year+month+day)][:]))
        else:
            obs_westerly[int(year+month+day)] = []
         #   obs_easterly[int(year+month+day)] = []
            obs_easterly[int(year+month+day)] = obs_easterly_idx[int(year+month+day)]

            for i in np.arange(0,np.array(sub_idx).shape[0]+1):                # westerlies and easterlies interchange daily
                if i == 0:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [ sub_idx[0]] - \
                                         obs_westerly_idx[int(year+month+day)] [0])
                elif i == np.array(sub_idx).shape[0]:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [-1] - \
                                         obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    obs_westerly_duration = (obs_westerly_idx[int(year+month+day)] [ sub_idx[i]] - \
                                         obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1])
                if obs_westerly_duration >= 28:
                    if i == 0:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
            #            print(obs_westerly_duration, RR_obs_westerly)
                    elif i == np.array(sub_idx).shape[0]:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
             #           print(obs_westerly_duration, RR_obs_westerly)
                    else:
                        obs_westerly[int(year+month+day)] = np.concatenate((obs_westerly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
              #          print(obs_westerly_duration, RR_obs_westerly)
                elif obs_westerly_duration < 28:
                    if i == 0:
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)], 
                                                                        obs_westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                   # print(obs_westerly_duration)
                        obs_easterly[int(year+month+day)] = np.concatenate((obs_easterly[int(year+month+day)],
                                                                        obs_westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
        

        if len(obs_westerly[int(year+month+day)]) == 0:    # only easterlies
            obs_easterly[int(year+month+day)]  = obs_easterly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
   #                        np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:]),2 ),
    #              'obs_easterly [mm/24h]:', 
     #             np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:]),2),
      #            'obs_westerly [mm/24h]:', 
       #           np.round(len(obs_westerly[int(year+month+day)]),2))
        elif len(obs_easterly[int(year+month+day)]) == 0:  # only westerlies
            obs_westerly[int(year+month+day)]  = obs_westerly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
   #                        np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:]),2 ),
    #              'obs_easterly [mm/24h]:', 
     #             np.round(len(obs_easterly[int(year+month+day)]),2),
      #            'obs_westerly [mm/24h]:', 
       #           np.round(np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:]),2))
        else:    # both easterlies and westerlies
            obs_easterly[int(year+month+day)]  = obs_easterly[int(year+month+day)].astype(int)
            obs_westerly[int(year+month+day)]  = obs_westerly[int(year+month+day)].astype(int)
            
#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
   #                        np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:]),2 ),
    #              'obs_easterly [mm/24h]:', 
     #             np.round(np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:]),2),
      #            'obs_westerly [mm/24h]:', 
       #           np.round(np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:]),2))
            
        # assign sum values to days
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
            obs_total_precip[int(year+month+day)] = np.nan
            obs_precip_east[int(year+month+day)] =  np.nan
            obs_precip_west[int(year+month+day)] =  np.nan
        else:
            obs_total_precip[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly_idx[int(year+month+day)],:]) + \
                                                np.nansum(obs_diff[int(year+month+day)][obs_westerly_idx[int(year+month+day)],:])
            obs_precip_east[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_easterly[int(year+month+day)],:])
            obs_precip_west[int(year+month+day)] = np.nansum(obs_diff[int(year+month+day)][obs_westerly[int(year+month+day)],:])
            
#        print([int(year+month+day)], 'total:', np.round(obs_total_precip[int(year+month+day)],2),
 #                                       'westerly:',  np.round(obs_precip_west[int(year+month+day)],2),
  #                                       'easterly:', np.round(obs_precip_east[int(year+month+day)],2) )
        
        

        #### assign nan values where air temperature is larger than 2 degC
        obs_diff_2deg[int(year+month+day)] = np.empty(shape=obs_diff[int(year+month+day)].shape)
        obs_diff_2deg[int(year+month+day)][:] = obs_diff[int(year+month+day)][:]

        obs_diff_2deg[int(year+month+day)][np.where(obs_air_temp_rolling[int(year+month+day)] > 2.),:] = np.nan

        # assign sum values to days with occuring temp < 2degC
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
            obs_total_precip_2deg[int(year+month+day)] = np.nan
            obs_precip_east_2deg[int(year+month+day)] =  np.nan
            obs_precip_west_2deg[int(year+month+day)] =  np.nan
        else:
            obs_total_precip_2deg[int(year+month+day)] = np.nansum(obs_diff_2deg[int(year+month+day)][obs_easterly[int(year+month+day)],:]) + \
                                                         np.nansum(obs_diff_2deg[int(year+month+day)][obs_westerly[int(year+month+day)],:])
            obs_precip_east_2deg[int(year+month+day)] = np.nansum(obs_diff_2deg[int(year+month+day)][obs_easterly[int(year+month+day)],:])
            obs_precip_west_2deg[int(year+month+day)] = np.nansum(obs_diff_2deg[int(year+month+day)][obs_westerly[int(year+month+day)],:])
        
        print([int(year+month+day)], np.round(obs_precip_west[int(year+month+day)],2), 
                                     np.round(obs_precip_east[int(year+month+day)],2), 
                                     np.round(obs_total_precip[int(year+month+day)],2))
        print([int(year+month+day)], np.round(obs_precip_west_2deg[int(year+month+day)],2), 
                                     np.round(obs_precip_east_2deg[int(year+month+day)],2), 
                                     np.round(obs_total_precip_2deg[int(year+month+day)],2))
        

### Eklima data

In [ ]:
eklima_f = dict()
eklima_double_fence_acc = dict()
eklima_wind_dir_10m   = dict()
eklima_wind_speed_10m = dict()
eklima_diff = dict()
eklima_westerly_idx = dict()
eklima_easterly_idx = dict()
eklima_westerly = dict()
eklima_easterly = dict()
eklima_total_precip = dict()
eklima_precip_west  = dict()
eklima_precip_east  = dict()
eklima_air_temp = dict()

eklima_diff_2deg = dict()
eklima_total_precip_2deg = dict()
eklima_precip_east_2deg = dict()
eklima_precip_west_2deg = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        eklima_dirnc = '../../Data/Eklima/%s/precip_wind_temp_%s%s%s.nc' %(station,year,month,day)
        eklima_f[int(year+month+day)] = netCDF4.Dataset(eklima_dirnc, 'r')

        eklima_double_fence_acc[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['RR_1'][:])
        eklima_wind_dir_10m[int(year+month+day)]   = (eklima_f[int(year+month+day)].variables['DD'][:])
        eklima_wind_speed_10m[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['FF'][:])
        eklima_air_temp[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['TA'][:])
        
        eklima_f[int(year+month+day)].close
        

        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        eklima_diff[int(year+month+day)] = np.empty(shape=(eklima_double_fence_acc[int(year+month+day)].shape))#/60))
        eklima_diff[int(year+month+day)][:] =  np.nan

        eklima_diff[int(year+month+day)][:,] = eklima_double_fence_acc[int(year+month+day)]

        # Find wind regimes and associated precip amount
        IDX = np.arange(eklima_wind_dir_10m[int(year+month+day)].shape[0])

        ## find all eklima_westerly regimes (202.5 - 22.5 deg) 
        eklima_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 202.5),
                                                                                     (eklima_wind_dir_10m[int(year+month+day)][:,] <= 360)),
                                                                      (eklima_wind_dir_10m[int(year+month+day)][:,] <22.5))]

        ## find all eklima_easterly regimes (22.5 - 202.5 deg)
        eklima_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 22.5),
                                                                       (eklima_wind_dir_10m[int(year+month+day)][:,] < 202.5))]

        eklima_westerly[int(year+month+day)] = eklima_westerly_idx[int(year+month+day)]
        eklima_easterly[int(year+month+day)] = eklima_easterly_idx[int(year+month+day)]

        if len(eklima_westerly[int(year+month+day)]) == 0:    # only easterlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(len(eklima_westerly[int(year+month+day)]),2))

        elif len(eklima_easterly[int(year+month+day)]) == 0:  # only westerlies
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)

#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(len(eklima_easterly[int(year+month+day)]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],]),2))
        else:    # both easterlies and westerlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)
            
#            print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 
  #                np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
   #                        np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],]),2 ),
    #              'eklima_easterly [mm/24h]:', 
     #             np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],]),2),
      #            'eklima_westerly [mm/24h]:', 
       #           np.round(np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],]),2))

        # assign sum values to days
        if len(eklima_easterly[int(year+month+day)]) == 0 and len(eklima_westerly[int(year+month+day)]) == 0:
            eklima_total_precip[int(year+month+day)] = np.nan
            eklima_precip_east[int(year+month+day)] = np.nan
            eklima_precip_west[int(year+month+day)] = np.nan
        else:
            eklima_total_precip[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_easterly_idx[int(year+month+day)],]) + \
                                           np.nansum(eklima_diff[int(year+month+day)][eklima_westerly_idx[int(year+month+day)],])
            eklima_precip_east[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_easterly[int(year+month+day)],])
            eklima_precip_west[int(year+month+day)] = np.nansum(eklima_diff[int(year+month+day)][eklima_westerly[int(year+month+day)],])
        

        #### assign nan values where air temperature is larger than 2 degC
        eklima_diff_2deg[int(year+month+day)] = np.empty(shape=eklima_diff[int(year+month+day)].shape)
        eklima_diff_2deg[int(year+month+day)][:] = eklima_diff[int(year+month+day)][:]
        eklima_diff_2deg[int(year+month+day)][np.where(eklima_air_temp[int(year+month+day)] > 2.),] = np.nan
        # assign sum values to days with occuring temp < 2degC
        if len(eklima_easterly[int(year+month+day)]) == 0 and len(eklima_westerly[int(year+month+day)]) == 0:
            eklima_total_precip_2deg[int(year+month+day)] = np.nan
            eklima_precip_east_2deg[int(year+month+day)] =  np.nan
            eklima_precip_west_2deg[int(year+month+day)] =  np.nan
        else:
            eklima_total_precip_2deg[int(year+month+day)] = np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_easterly[int(year+month+day)],]) + \
                                                         np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_westerly[int(year+month+day)],])
            eklima_precip_east_2deg[int(year+month+day)] = np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_easterly[int(year+month+day)],])
            eklima_precip_west_2deg[int(year+month+day)] = np.nansum(eklima_diff_2deg[int(year+month+day)][eklima_westerly[int(year+month+day)],])
        

### assign eklima data where total precip < 0
        if obs_total_precip[int(year+month+day)] < 0:
            if obs_precip_east[int(year+month+day)] < 0 and obs_precip_west[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
                obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
    
            elif obs_precip_east[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
                obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
        
            elif obs_precip_west[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
                obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
        
        elif obs_precip_east[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
            obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])   
    
        elif obs_precip_west[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
            obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
        elif np.isnan(obs_total_precip[int(year+month+day)]):
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
            obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#        print([int(year+month+day)], 'observ total:', np.round(obs_total_precip[int(year+month+day)],2),
 #                                       'westerly:',  np.round(obs_precip_west[int(year+month+day)],2),
  #                                       'easterly:', np.round(obs_precip_east[int(year+month+day)],2) )



### assign eklima data where total precip_2deg < 0
        if obs_total_precip_2deg[int(year+month+day)] < 0:
            if obs_precip_east_2deg[int(year+month+day)] < 0 and obs_precip_west_2deg[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
                obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'all are negative',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)],obs_precip_west[int(year+month+day)])
    
            elif obs_precip_east_2deg[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
                obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'total and east',obs_total_precip[int(year+month+day)],obs_precip_east[int(year+month+day)])
        
            elif obs_precip_west_2deg[int(year+month+day)] < 0:
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
                obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
                obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
                obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
                obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
                obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
                obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
                obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
                obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
                obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
                obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
                obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
                obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
                obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#                print([int(year+month+day)], 'total and west',obs_total_precip[int(year+month+day)],obs_precip_west[int(year+month+day)])
        
        elif obs_precip_east_2deg[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
            obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#            print([int(year+month+day)], 'only east',obs_precip_east[int(year+month+day)])   
    
        elif obs_precip_west_2deg[int(year+month+day)] < 0:
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
            obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
#            print([int(year+month+day)], 'only west',obs_precip_west[int(year+month+day)])
        elif np.isnan(obs_total_precip_2deg[int(year+month+day)]):
            obs_total_precip[int(year+month+day)] = eklima_total_precip[int(year+month+day)]
            obs_precip_east[int(year+month+day)] = eklima_precip_east[int(year+month+day)]
            obs_precip_west[int(year+month+day)] = eklima_precip_west[int(year+month+day)]
            obs_wind_speed_rolling[int(year+month+day)] = eklima_wind_speed_10m[int(year+month+day)]
            obs_wind_dir_rolling[int(year+month+day)] = eklima_wind_dir_10m[int(year+month+day)]
            obs_air_temp_rolling[int(year+month+day)] = eklima_air_temp[int(year+month+day)]
            obs_diff[int(year+month+day)] = eklima_diff[int(year+month+day)]
            obs_easterly[int(year+month+day)] = eklima_easterly[int(year+month+day)]
            obs_westerly[int(year+month+day)] = eklima_westerly[int(year+month+day)]
                
            obs_total_precip_2deg[int(year+month+day)] = eklima_total_precip_2deg[int(year+month+day)]
            obs_precip_east_2deg[int(year+month+day)] = eklima_precip_east_2deg[int(year+month+day)]
            obs_precip_west_2deg[int(year+month+day)] = eklima_precip_west_2deg[int(year+month+day)]
            obs_diff_2deg[int(year+month+day)] = eklima_diff_2deg[int(year+month+day)]
            
        print([int(year+month+day)], np.round(obs_precip_west[int(year+month+day)],2), 
                                     np.round(obs_precip_east[int(year+month+day)],2), 
                                     np.round(obs_total_precip[int(year+month+day)],2))
        print([int(year+month+day)], np.round(obs_precip_west_2deg[int(year+month+day)],2), 
                                     np.round(obs_precip_east_2deg[int(year+month+day)],2), 
                                     np.round(obs_total_precip_2deg[int(year+month+day)],2))

### MEPS data

In [ ]:
def fill_values(variable):
    if np.ma.is_masked(variable):
        mask = np.ma.getmaskarray(variable[:,:,:])
        marr = np.ma.array(variable[:,:,:], mask = mask, fill_value = np.nan)
        filled = marr.filled(np.nan)
        
    else:
        filled = variable[:]
        
    return(filled)

In [ ]:
meps_f = dict()
meps_precip_acc = dict()
meps_xwind_10m = dict()
meps_ywind_10m = dict()
meps_diff = dict()
meps_wind_dir   = dict()
meps_wind_speed = dict()
meps_westerly_idx = dict()
meps_easterly_idx = dict()
meps_westerly = dict()
meps_easterly = dict()
meps_total_precip = dict()
meps_precip_west  = dict()
meps_precip_east  = dict()
meps_air_temperature_2m = dict()
meps_diff_2deg = dict()
meps_total_precip_2deg = dict()
meps_precip_east_2deg = dict()
meps_precip_west_2deg = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        meps_dirnc = '../../Data/MEPS/%s/%s%s/sfc_hybrid5_allEM_00/%s%s%s_00.nc' %(station,year,month,year,month,day)
        try:
            meps_f[int(year+month+day)] = netCDF4.Dataset(meps_dirnc, 'r')
        except OSError:
            print('no file found: %s' %meps_dirnc)
#            meps_precip_acc[int(year+month+day)] = np.empty(shape=(67, 1, 10))
 #           meps_xwind_10m[int(year+month+day)] = np.empty(shape=(67, 1, 10))
  #          meps_ywind_10m[int(year+month+day)] = np.empty(shape=(67, 1, 10))
            
            meps_total_precip[int(year+month+day)] = np.empty(shape=(10, ))
            meps_total_precip[int(year+month+day)][:] = np.nan
            meps_precip_east[int(year+month+day)] = np.empty(shape=(10, ))
            meps_precip_east[int(year+month+day)][:] = np.nan
            meps_precip_west[int(year+month+day)] = np.empty(shape=(10, ))
            meps_precip_west[int(year+month+day)][:] = np.nan
            
            meps_wind_dir[int(year+month+day)] = np.empty(shape=(25, 1, 10))
            meps_wind_dir[int(year+month+day)][:] = np.nan
            meps_wind_speed[int(year+month+day)] =  np.empty(shape=(25, 1, 10))
            meps_wind_speed[int(year+month+day)][:] = np.nan
            meps_westerly_idx[int(year+month+day)] = np.empty(shape=(10,))
            meps_westerly_idx[int(year+month+day)][:] = np.nan
            meps_easterly_idx[int(year+month+day)] = np.empty(shape=(10,))
            meps_easterly_idx[int(year+month+day)][:] = np.nan
            meps_diff[int(year+month+day)] = np.empty(shape=(25, 1, 10))
            meps_diff[int(year+month+day)][:] = np.nan
            meps_diff_2deg[int(year+month+day)] = np.empty(shape=(25, 1, 10))
            meps_diff_2deg[int(year+month+day)][:] = np.nan
            
            continue

        meps_precip_acc[int(year+month+day)] = (meps_f[int(year+month+day)].variables['precipitation_amount_acc'][:])
        meps_xwind_10m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['x_wind_10m'][:])
        meps_ywind_10m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['y_wind_10m'][:])
        meps_air_temperature_2m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['air_temperature_2m'][:])
        
        ## substitute missing values with nan
        meps_precip_acc[int(year+month+day)] = fill_values(meps_precip_acc[int(year+month+day)][:,:,:])
        meps_xwind_10m[int(year+month+day)] = fill_values(meps_xwind_10m[int(year+month+day)][:,:,:])
        meps_ywind_10m[int(year+month+day)] = fill_values(meps_ywind_10m[int(year+month+day)][:,:,:])
        meps_air_temperature_2m[int(year+month+day)] = fill_values(meps_air_temperature_2m[int(year+month+day)][:,:,:])
        
#        meps_wind_dir_10m[int(year+month+day)]   = (meps_f[int(year+month+day)].variables['10m_wind_dir_mast1'][:])
#        meps_wind_speed_10m[int(year+month+day)] = (meps_f[int(year+month+day)].variables['10m_wind_speed_mast1'][:])
        
        meps_f[int(year+month+day)].close
        

        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        meps_diff[int(year+month+day)] = np.empty(shape=(meps_precip_acc[int(year+month+day)].shape))#/60))
        meps_diff[int(year+month+day)][:] =  np.nan

        for i in np.arange(0,meps_precip_acc[int(year+month+day)].shape[0]-1):
            meps_diff[int(year+month+day)][int(i),:,:] = meps_precip_acc[int(year+month+day)][int(i)+1,:,:] - \
                                                         meps_precip_acc[int(year+month+day)][int(i),:,:]

        #### calculate wind direction / speed
        meps_wind_dir[int(year+month+day)] = np.arctan2(meps_ywind_10m[int(year+month+day)][:,:,:], 
                                                        meps_xwind_10m[int(year+month+day)][:,:,:]) 
        meps_wind_dir[int(year+month+day)] = 270 - np.rad2deg(meps_wind_dir[int(year+month+day)])
        larger360 = np.where(meps_wind_dir[int(year+month+day)][:,:,:] > 360.)
        meps_wind_dir[int(year+month+day)][larger360] = meps_wind_dir[int(year+month+day)][larger360]-360
    
        meps_wind_speed[int(year+month+day)] = np.sqrt((meps_xwind_10m[int(year+month+day)][:,:,:])**2 + \
                                                           (meps_ywind_10m[int(year+month+day)][:,:,:])**2 )
        
        ### only get values up to 24h
        meps_diff[int(year+month+day)] = meps_diff[int(year+month+day)][:25,:,:]
        meps_wind_dir[int(year+month+day)] = meps_wind_dir[int(year+month+day)][:25,:,:]
        meps_wind_speed[int(year+month+day)] = meps_wind_speed[int(year+month+day)][:25,:,:]
        meps_air_temperature_2m[int(year+month+day)] = meps_air_temperature_2m[int(year+month+day)][:25,:,:] # in Kelvin


        # Find wind regimes and associated precip amount
        IDX = np.arange(meps_wind_dir[int(year+month+day)].shape[0])

        meps_westerly_idx[int(year+month+day)] = []
        meps_easterly_idx[int(year+month+day)] = []

        for ens_memb in range(10):
            ## find all meps_westerly regimes (202.5 - 22.5 deg) 
            meps_westerly_idx[int(year+month+day)].append(IDX[np.logical_or(
                    np.logical_and((meps_wind_dir[int(year+month+day)][:,0,ens_memb] >= 202.5), 
                                   (meps_wind_dir[int(year+month+day)][:,0,ens_memb] <= 360)),
                    (meps_wind_dir[int(year+month+day)][:,0,ens_memb] <22.5))])
#        print(np.array(meps_westerly_idx[int(year+month+day)][ens_memb]).shape)


            ## find all meps_easterly regimes (22.5 - 202.5 deg)
            meps_easterly_idx[int(year+month+day)].append(IDX[np.logical_and(
                    (meps_wind_dir[int(year+month+day)][:,0,ens_memb] >= 22.5),
                    (meps_wind_dir[int(year+month+day)][:,0,ens_memb] < 202.5))])
#        print(np.array(meps_easterly_idx[int(year+month+day)][ens_memb]).shape)

        
        meps_westerly[int(year+month+day)] = meps_westerly_idx[int(year+month+day)]
        meps_easterly[int(year+month+day)] = meps_easterly_idx[int(year+month+day)]
        
        meps_easterly_idx[int(year+month+day)] = np.array(meps_easterly_idx[int(year+month+day)])
        meps_westerly_idx[int(year+month+day)] = np.array(meps_westerly_idx[int(year+month+day)])

        meps_total_precip[int(year+month+day)] = []
        meps_precip_east[int(year+month+day)] = []
        meps_precip_west[int(year+month+day)] = []

        for ens_memb in range(10):
#            if len(meps_westerly[int(year+month+day)][ens_memb]) == 0:    # only easterlies
#                print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 'EM=%s' %ens_memb,
  #                np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
   #                        np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2 ),
    #              'meps_easterly [mm/24h]:', 
     #             np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2),
      #            'meps_westerly [mm/24h]:', 
       #           np.round(len(meps_westerly[int(year+month+day)][ens_memb]),2))

#            elif len(meps_easterly[int(year+month+day)][ens_memb]) == 0:  # only westerlies
#                print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 'EM=%s' %ens_memb,
  #                np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
   #                        np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2 ),
    #              'meps_easterly [mm/24h]:', 
     #             np.round(len(meps_easterly[int(year+month+day)][ens_memb]),2),
      #            'meps_westerly [mm/24h]:', 
       #           np.round(np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2))

#            else:    # both easterlies and westerlies
#                print([int(year+month+day)], 
 #                 'total precipitation [mm/24h]:', 'EM=%s' %ens_memb,
  #                np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
   #                        np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2 ),
    #              'meps_easterly [mm/24h]:', 
     #             np.round(np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2),
      #            'meps_westerly [mm/24h]:', 
       #           np.round(np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb]),2))
            

            # assign sum values to days
            total = np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
                    np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb])
            meps_total_precip[int(year+month+day)].append(total)
    
            easterly = np.nansum(meps_diff[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb])
            meps_precip_east[int(year+month+day)].append(easterly)
    
            westerly = np.nansum(meps_diff[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb])
            meps_precip_west[int(year+month+day)].append(westerly)
            
            
        #### assign nan values where air temperature is larger than 2 degC
        meps_total_precip_2deg[int(year+month+day)] = [] 
        meps_precip_east_2deg[int(year+month+day)] = []
        meps_precip_west_2deg[int(year+month+day)] = []
        
        meps_diff_2deg[int(year+month+day)] = np.empty(shape=meps_diff[int(year+month+day)].shape)
        meps_diff_2deg[int(year+month+day)][:] = meps_diff[int(year+month+day)][:]

        for ens_memb in range(10):
            mepsT_2deg = (np.where(meps_air_temperature_2m[int(year+month+day)][:,0,ens_memb] > 275.15))
            meps_diff_2deg[int(year+month+day)][mepsT_2deg,:,ens_memb] = np.nan
 #       # assign sum values to days with occuring temp < 2degC
            if len(meps_easterly_idx[int(year+month+day)][ens_memb]) == 0 and \
               len(meps_westerly_idx[int(year+month+day)][ens_memb]) == 0:
                meps_total_precip_2deg[int(year+month+day)].append(np.nan)
                meps_precip_east_2deg[int(year+month+day)].append(np.nan)
                meps_precip_west_2deg[int(year+month+day)].append(np.nan)
            else:
                total = np.nansum(meps_diff_2deg[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb]) + \
                        np.nansum(meps_diff_2deg[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb])
                meps_total_precip_2deg[int(year+month+day)].append(total)

                easterly = np.nansum(meps_diff_2deg[int(year+month+day)][meps_easterly_idx[int(year+month+day)][ens_memb],:,ens_memb])
                meps_precip_east_2deg[int(year+month+day)].append(easterly)

                westerly = np.nansum(meps_diff_2deg[int(year+month+day)][meps_westerly_idx[int(year+month+day)][ens_memb],:,ens_memb])
                meps_precip_west_2deg[int(year+month+day)].append(westerly)
       

### Retrieval data

In [ ]:
ret_f = dict()
ret_accumulation = dict()
ret_total_precip = dict()
ret_precip_east = dict()
ret_precip_west = dict()
ret_acc_24 = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
        
        ret_dirnc = '../../Data/Retrieval/%s/combined/precip_wind_temp_%s%s%s.nc' %(station,year,month,day)
        ret_f[int(year+month+day)] = netCDF4.Dataset(ret_dirnc, 'r')

        ret_accumulation[int(year+month+day)] = (ret_f[int(year+month+day)].variables['precip_amount'][:])

        # assign sum values to days
        ret_total_precip[int(year+month+day)] = ret_f[int(year+month+day)].variables['24h_precip_sum'][:]
        ret_precip_east[int(year+month+day)] = ret_f[int(year+month+day)].variables['24h_upslope_sum'][:]
        ret_precip_west[int(year+month+day)] = ret_f[int(year+month+day)].variables['24h_pulsed_sum'][:]
        
        if len(ret_accumulation[int(year+month+day)]) != len(obs_diff[int(year+month+day)]):
           # print(int(year+month+day), len(ret_accumulation[int(year+month+day)]), len(obs_diff[int(year+month+day)]))
            ret_acc_24[int(year+month+day)] = np.empty(shape=(int(ret_accumulation[int(year+month+day)].shape[0]/60),))
            ret_acc_24[int(year+month+day)][:] = np.nan
            for i in range(int(ret_accumulation[int(year+month+day)].shape[0]/60)):
                ret_acc_24[int(year+month+day)][i] = np.sum(ret_accumulation[int(year+month+day)][i*60:(i+1)*60])

            ret_accumulation[int(year+month+day)] = np.empty(shape=(int(ret_accumulation[int(year+month+day)].shape[0]/60),))
            ret_accumulation[int(year+month+day)][:] = ret_acc_24[int(year+month+day)][:]

#############################        
        ## set days, still with not exisiting observation data to 0 mm precipitation
        if np.isnan(obs_total_precip[int(year+month+day)]):
#            print([int(year+month+day)], 'total precip is NAN')
            obs_total_precip[int(year+month+day)] = 0.0
            meps_total_precip[int(year+month+day)] = np.zeros(shape=(10,))
            ret_total_precip[int(year+month+day)] = 0.0
        if np.isnan(obs_precip_east[int(year+month+day)]):
 #           print([int(year+month+day)], 'easterly is NAN')
            obs_precip_east[int(year+month+day)] = 0.0
            meps_precip_east[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_east[int(year+month+day)] = 0.0
        if np.isnan(obs_precip_west[int(year+month+day)]):
  #          print([int(year+month+day)], 'westerly is NAN')
            obs_precip_west[int(year+month+day)] = 0.0
            meps_precip_west[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_west[int(year+month+day)] = 0.0
        


        ## set days with less than 24h recorded values to 0mm precipitation 
        if np.count_nonzero(~np.isnan(obs_wind_dir_rolling[int(year+month+day)][:])) != 1440 and \
           np.count_nonzero(~np.isnan(obs_wind_dir_rolling[int(year+month+day)][:])) != 24:
   #         print([int(year+month+day)], 'less than 24h observation')
            obs_total_precip[int(year+month+day)] = 0.0
            meps_total_precip[int(year+month+day)] = np.zeros(shape=(10,))
            ret_total_precip[int(year+month+day)] = 0.0
            
            obs_total_precip_2deg[int(year+month+day)] = 0.0
            meps_total_precip_2deg[int(year+month+day)]= np.zeros(shape=(10,))
            
            obs_precip_east[int(year+month+day)] = 0.0
            meps_precip_east[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_east[int(year+month+day)] = 0.0
            
            obs_precip_east_2deg[int(year+month+day)] = 0.0
            meps_precip_east_2deg[int(year+month+day)] = np.zeros(shape=(10,))
            
            obs_precip_west[int(year+month+day)] = 0.0
            meps_precip_west[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_west[int(year+month+day)] = 0.0
            
            obs_precip_west_2deg[int(year+month+day)] = 0.0
            meps_precip_west_2deg[int(year+month+day)] = np.zeros(shape=(10,))
        
        
        

        if obs_total_precip[int(year+month+day)]< 0.25:
    #        print([int(year+month+day)], 'less than 0.25 mm precipitation')
            obs_total_precip[int(year+month+day)] = 0.0
            meps_total_precip[int(year+month+day)] = np.zeros(shape=(10,))
            ret_total_precip[int(year+month+day)] = 0.0
            
            obs_total_precip_2deg[int(year+month+day)] = 0.0
            meps_total_precip_2deg[int(year+month+day)] = np.zeros(shape=(10,))
            
            obs_precip_east[int(year+month+day)] = 0.0
            meps_precip_east[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_east[int(year+month+day)] = 0.0
            
            obs_precip_east_2deg[int(year+month+day)] = 0.0
            meps_precip_east_2deg[int(year+month+day)] = np.zeros(shape=(10,))
            
            obs_precip_west[int(year+month+day)] = 0.0
            meps_precip_west[int(year+month+day)] = np.zeros(shape=(10,))
            ret_precip_west[int(year+month+day)] = 0.0
            
            obs_precip_west_2deg[int(year+month+day)] = 0.0
            meps_precip_west_2deg[int(year+month+day)] = np.zeros(shape=(10,))
            
#        print([int(year+month+day)], 'east', np.round(obs_precip_east_2deg[int(year+month+day)],2),
 #                                    'west', np.round(obs_precip_west_2deg[int(year+month+day)],2),
  #                                   'total', np.round(obs_total_precip_2deg[int(year+month+day)],2))
   #     
        
       

In [ ]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [ ]:
nc_dir = '../../Data/Weathermast_MEPS_Retrieval/'

In [ ]:

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
            
        meps_westerly_idx_array = np.empty(shape=meps_wind_dir[int(year+month+day)][:,0,:].shape)
        meps_westerly_idx_array[:] = np.nan

        meps_easterly_idx_array = np.empty(shape=meps_wind_dir[int(year+month+day)][:,0,:].shape)
        meps_easterly_idx_array[:] = np.nan
        if int(year+month+day) == 20161216 or int(year+month+day) == 20170101:
            print(int(year+month+day), 'no MEPS forecasts available.')
        else:
            for ens_memb in range(10):
                meps_westerly_idx_array[meps_westerly_idx[int(year+month+day)][ens_memb],ens_memb] = meps_westerly_idx[int(year+month+day)][ens_memb]
                meps_easterly_idx_array[meps_easterly_idx[int(year+month+day)][ens_memb],ens_memb] = meps_easterly_idx[int(year+month+day)][ens_memb]

            ### write netCDF file
            cF.createFolder(nc_dir+station)
            fw = netCDF4.Dataset(nc_dir+station+'/weathermast_meps_retrieval_%s%s%s.nc' %(year,month,day), 'w')
            ### create dimensions
            fw.createDimension('time_obs', obs_wind_dir_rolling[int(year+month+day)].shape[0] )
            fw.createDimension('time_meps', meps_wind_dir[int(year+month+day)].shape[0])
            fw.createDimension('ensemble_member', 10)
            fw.createDimension('obs_idx_west', np.array(obs_westerly[int(year+month+day)]).shape[0])
            fw.createDimension('obs_idx_east', np.array(obs_easterly[int(year+month+day)]).shape[0])

            obs_wd_w       = get_netCDF_variable(fw, 'obs_wind_direction', obs_wind_dir_rolling[int(year+month+day)], ('time_obs',))
            obs_ws_w       = get_netCDF_variable(fw, 'obs_wind_speed', obs_wind_speed_rolling[int(year+month+day)], ('time_obs',))
            obs_diff_w     = get_netCDF_variable(fw, 'obs_precipitation_difference', obs_diff[int(year+month+day)], ('time_obs',))
            obs_diff_2d_w  = get_netCDF_variable(fw, 'obs_precipitation_difference_2degC', obs_diff_2deg[int(year+month+day)], ('time_obs',))
            obs_west_w     = get_netCDF_variable(fw, 'obs_idx_westerly', np.array(obs_westerly[int(year+month+day)]), ('obs_idx_west',))
            obs_east_w     = get_netCDF_variable(fw, 'obs_idx_easterly', np.array(obs_easterly[int(year+month+day)]), ('obs_idx_east',))

            meps_wd_w      = get_netCDF_variable(fw, 'meps_wind_direction', meps_wind_dir[int(year+month+day)][:,0,:], ('time_meps', 'ensemble_member',))
            meps_ws_w      = get_netCDF_variable(fw, 'meps_wind_speed', meps_wind_speed[int(year+month+day)][:,0,:], ('time_meps', 'ensemble_member',))
            meps_diff_w    = get_netCDF_variable(fw, 'meps_precipitation_difference', meps_diff[int(year+month+day)][:,0,:], ('time_meps', 'ensemble_member',))
            meps_diff_2d_w = get_netCDF_variable(fw, 'meps_precipitation_difference_2degC', meps_diff_2deg[int(year+month+day)][:,0,:], ('time_meps', 'ensemble_member',))
            meps_west_w    = get_netCDF_variable(fw, 'meps_idx_westerly', meps_westerly_idx_array[:], ('time_meps', 'ensemble_member',))
            meps_east_w    = get_netCDF_variable(fw, 'meps_idx_easterly', meps_easterly_idx_array[:], ('time_meps', 'ensemble_member',))

            ret_diff_w     = get_netCDF_variable(fw, 'ret_precipitation_difference', ret_accumulation[int(year+month+day)], ('time_obs',))

            fw.close()
            print('file written:', nc_dir+station+'/weathermast_meps_retrieval_%s%s%s.nc' %(year,month,day))